In [2]:
from resnet_variant import *
from baseline_cnn import *
from resnet_variant import resnet_n
import torch.nn as nn
import torch.optim as optim
import time
import pathlib
import torch
import torch.nn.functional as F
from evaluation import Evaluation
from xray_imbalanced_dataloader import create_balanced_split_loaders
import random
%load_ext autoreload
%autoreload 2
# Setup: initialize the hyperparameters/variables
num_epochs = 1           # Number of full passes through the dataset
batch_size = 16          # Number of samples in each minibatch
learning_rate = 0.001  
seed = np.random.seed(1) # Seed the random number generator for reproducibility
p_val = 0.1              # Percent of the overall dataset to reserve for validation
p_test = 0.2             # Percent of the overall dataset to reserve for testing

class channelCopy(object):    
    def __call__(self, img):
        return torch.cat([img, img, img], 0)
    
def augmentImg(img): 
    if 0.9 < random.random():
        return img

    t=transforms.RandomRotation(10)
    img = t(img)
    return img
        
#TODO: Convert to Tensor - you can later add other transformations, such as Scaling here
transform = transforms.Compose([transforms.Resize(512),augmentImg,transforms.ToTensor(),channelCopy()])


# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

# Setup the training, validation, and testing dataloaders
train_loader, val_loader, test_loader = create_split_loaders(batch_size, seed, transform=transform, 
                                                             p_val=p_val, p_test=p_test,
                                                             shuffle=True, show_sample=False, 
                                                             extras=extras)
# train_loader, val_loader, test_loader = create_balanced_split_loaders(batch_size, seed, transform=transform,
#                                                                          p_val=p_val, p_test=p_test,
#                                                                          shuffle=True, show_sample=False,
#                                                                          extras=extras, z_score=True)
# Instantiate a BasicCNN to run on the GPU or CPU based on CUDA support
model = resnet_n(pretrained=False, num_classes=14)
model = model.to(computing_device)
print("Model on CUDA?", next(model.parameters()).is_cuda)

#TODO: Define the loss criterion and instantiate the gradient descent optimizer
#criterion = nn.MultiLabelSoftMarginLoss() #TODO - loss criteria are defined in the torch.nn package
criterion = nn.MultiLabelSoftMarginLoss()
#TODO: Instantiate the gradient descent optimizer - use Adam optimizer with default parameters
optimizer = optim.Adam(model.parameters()) #TODO - optimizers are defined in the torch.optim package

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
CUDA NOT supported
Model on CUDA? False


In [ ]:
transform = transforms.Compose([transforms.Resize(512), transforms.ToTensor()])

In [ ]:
train_loader, val_loader, test_loader = create_split_loaders(batch_size, seed, transform=transform, 
                                                             p_val=p_val, p_test=p_test,
                                                             shuffle=True, show_sample=False, 
                                                             extras=extras)

In [ ]:
for minibatch_count, (images, labels) in enumerate(train_loader, 0):
    

In [ ]:
inbalance_weight = torch.Tensor([1/8236,1/1989,1/9561,1/14240,1/4175,1/4537,1/962,1/3825,1/3317,1/1637,1/1819,1/1237,1/2431,1/162])

In [ ]:
inbalance_weight

In [ ]:
inbalance_weight/torch.norm(inbalance_weight)

In [ ]:
F.normalize(inbalance_weight)

In [25]:
x = torch.rand((4,6))

In [35]:
y = torch.bernoulli(x)

In [36]:
x

tensor([[0.6626, 0.7595, 0.5292, 0.2148, 0.1666, 0.1843],
        [0.6687, 0.8035, 0.1697, 0.8229, 0.9938, 0.0686],
        [0.9343, 0.1650, 0.7778, 0.0114, 0.2583, 0.1929],
        [0.6283, 0.9699, 0.2832, 0.6697, 0.8875, 0.0431]])

In [37]:
y

tensor([[1., 0., 1., 0., 0., 0.],
        [1., 1., 0., 1., 1., 0.],
        [1., 0., 1., 0., 0., 0.],
        [0., 1., 1., 1., 1., 0.]])

In [ ]:
torch.sum(x,dim=0)

In [ ]:
x.size()[0]

In [46]:
torch.Tensor(5,5)/10

tensor([[1.5194e+29, 4.5893e-42, 1.5194e+29, 4.5893e-42, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]])

In [47]:
class W_CEL(nn.Module):
    def __init__(self, weight=None, Verbose=False):
        super(W_CEL, self).__init__()
        self.weight = weight 
        self.Verbose = Verbose
        
    def forward(self, input, target):
        class_sum = torch.sum(target, dim=0)
        
        positive_ratio = class_sum / target.size()[0]   # P / (P+N)
        print(target.size()[0])
        negative_ratio = torch.ones(target.size()) - positive_ratio # N / (P+N)
        print(positive_ratio)
        loss = -(target * F.logsigmoid(input) * negative_ratio + (1 - target) * F.logsigmoid(-input) * positive_ratio)
        
        loss = loss.sum(dim=1) / input.size(1)
#         return loss
        ret = loss.mean()
        
        return ret

In [48]:
w = W_CEL()

In [49]:
w.forward(x,y)

4
tensor([0.7500, 0.5000, 0.7500, 0.5000, 0.5000, 0.0000])


tensor(0.2334)

In [ ]:
{'Atelectasis': 8236, 'Effusion': 9561, 'Infiltration': 14240, 'Mass': 4175, 'Nodule': 4537, 'No Finding': 43579, 'Consolidation': 3317, 'Pneumothorax': 3825, 'Pleural_Thickening': 2431, 'Emphysema': 1819, 'Cardiomegaly': 1989, 'Pneumonia': 962, 'Fibrosis': 1237, 'Edema': 1637, 'Hernia': 162}

In [ ]:
{0: "Atelectasis", 1: "Cardiomegaly", 2: "Effusion",
                        3: "Infiltration", 4: "Mass", 5: "Nodule", 6: "Pneumonia",
                        7: "Pneumothorax", 8: "Consolidation", 9: "Edema",
                        10: "Emphysema", 11: "Fibrosis",
                        12: "Pleural_Thickening", 13: "Hernia"}

In [ ]:
x

In [ ]:
x = torch.rand(2,3)

In [ ]:
Path = '/datasets/ChestXray-NIHCC/images'
paths = []
images = []

In [ ]:
import os
from PIL import Image

In [ ]:
for path in os.listdir(Path):
    if path == 'scale_images.py':
        continue
    paths.append(os.path.join(Path,path))

In [ ]:
mean = np.zeros((1024,1024))

In [ ]:
for path in paths:
    if path  == '/datasets/ChestXray-NIHCC/images/scale_images.py':
        print('haha')

In [ ]:
for i, path in enumerate(paths):
    
    meta = Image.open(path).convert(mode='L')
    image = np.array(meta) # a is readonly
    if image.shape == (1024,1024,4):
        print('4')
        continue
    mean += image
    
    if i % 5000 == 0:
        print(i)
        np.savetxt('tmp.txt',mean, fmt='%f') 
#     print(image.shape)
#     images.append(image)

In [ ]:
i

In [ ]:
mean.shape

In [ ]:
np.savetxt('mean.txt',mean, fmt='%f')

In [ ]:
len(images)

In [ ]:
std_meta = np.zeros((1024,1024))

In [ ]:
for i, path in enumerate(paths):
    
    meta = Image.open(path).convert(mode='L')
    image = np.array(meta) # a is readonly

    std_meta += (image-mean)**2
    
    if i % 5000 == 0:
        print(i)
        np.savetxt('std_meta.txt',std_meta, fmt='%f') 

In [ ]:
std = std_meta / 112119

In [ ]:
np.savetxt('std.txt',std, fmt='%f') 